In [ ]:
from typing import List
from copy import deepcopy
import numpy
import sys

In [ ]:
"""Provides a generic definition of the methods to create and use hyper-heuristics."""
class HyperHeuristic:

  def __init__(self, features : List[str], heuristics : List[str]):    
    self._features = deepcopy(features)
    self._heuristics = deepcopy(heuristics)

  def getFeatures(self) -> List[str]:    
    return deepcopy(self._features)

  def getHeuristics(self) -> List[str]:    
    return deepcopy(self._heuristics)

  

In [ ]:
"""Provides the basic functionality for all the problems supported by the system."""
class Problem:

  def solve(self, heuristic : str) -> None:
    """Solves this problem by using a specific heuristic."""
    raise Exception("Method not implemented yet.")

  def solveHH(self, hyperHeuristic : HyperHeuristic) -> None:
    """Solves this problem by using a hyper-heuristic."""
    raise Exception("Method not implemented yet.")

  def getFeature(self, feature : str) -> float:
    """Returns the current value of a given feature."""
    raise Exception("Method not implemented yet.")

  def getObjValue(self) -> float:
    """Returns the objective value of the current solution to this problem."""
    raise Exception("Method not implemented yet.")

def getHeuristic(self, problem : Problem) -> str:
    """Solves this problem by using a hyper-heuristic."""
    raise Exception("Method not implemented yet.")

HyperHeuristic.getHeuristic = getHeuristic

In [ ]:
"""Provides the methods to create and use items for the knapsack problem."""
class Item:
  
  def __init__(self, id, profit : int, weight: int):
    self.id = id
    self.profit = profit
    self.weight = weight
  
  def getId(self):
    return self.id

  def getProfit(self):
    return self.profit

  def getWeight(self):
    return self.weight

  def getProfitPerWeightUnit(self):
    return self.profit / self.weight

  def __str__(self):
    return f"({self.id}, {self.profit}, {self.profit})"

In [ ]:
"""Provides the methods to create and use knapsacks for the knapsack problem."""
class Knapsack:

  """Creates a new instance of Knapsack"""
  def __init__(self, capacity : int):    
    self._capacity = capacity
    self._profit = 0
    self._items = []    

  def getCapacity(self) -> int:
    return self._capacity
  
  def getProfit(self) -> int:
    return self._profit

  def canPack(self, item : Item) -> bool:
    return item.getWeight() <= (self._capacity)

  def pack(self, item : Item) -> None:
    if item.getWeight() <= self._capacity:
      self._items.append(item)
      self._capacity -= item.getWeight()
      self._profit += item.getProfit()
      return True
    return False
  
  def __str__(self):
    text = "("
    for item in self._items:
      text += str(item)
    text += ")"
    return text

In [ ]:
"""Provides the methods to create and solve knapsack problems."""
class KP (Problem):

  def __init__(self, fileName : str):
    f = open(fileName, "r")
    lines = f.readlines()
    line = lines[0].split(",")
    nbItems = int(line[0].strip())
    self._capacity = int(line[1].strip())
    self._items = [None] * nbItems
    for i in range(0, nbItems):
      line = lines[i + 1].split(",")
      weight = int(line[0].strip())
      profit = float(line[1].strip())
      self._items[i] = Item(i, weight, profit)    
    self._knapsack = Knapsack(self._capacity)
    
  def solve(self, heuristic : str) -> None:
    item = None    
    item = self._nextItem(heuristic)      
    while item != None:
      self._knapsack.pack(item)
      self._items.remove(item)
      item = self._nextItem(heuristic)
  
  def solveHH(self, hyperHeuristic : HyperHeuristic) -> None:
    item = None
    heuristic = hyperHeuristic.getHeuristic(self)
    item = self._nextItem(heuristic)
    while item != None:      
      self._knapsack.pack(item)
      self._items.remove(item)
      heuristic = hyperHeuristic.getHeuristic(self)
      item = self._nextItem(heuristic)    
  
  def getObjValue(self) -> float:
    return -self._knapsack.getProfit()

  def getFeature(self, feature : str) -> float:
    if feature == "MEAN_WEIGHT":
        values = [0] * len(self._items)
        for i in range(len(self._items)):
          values[i] = self._items[i].getWeight()
        return (sum(values) / len(values)) / max(values)
    else:
      raise Exception("Feature '" + feature + "' is not recognized by the system.")
   
  def _nextItem(self, heuristic : str) -> Item:    
    selected = None
    if heuristic == "DEFAULT":
        for item in self._items:
          if self._knapsack.canPack(item):
            selected = item
            break
        return selected
    if heuristic == "MAX_PROFIT":
        value = -sys.maxsize - 1
        for item in self._items:
          if self._knapsack.canPack(item) and item.getProfit() > value:
            selected = item
            value = item.getProfit()            
        return selected
    if heuristic == "MAX_PROFIT/WEIGHT":
        value = -sys.maxsize - 1
        for item in self._items:
          if self._knapsack.canPack(item) and item.getProfitPerWeightUnit() > value:
            selected = item
            value = item.getProfitPerWeightUnit()            
        return selected
    if heuristic == "MIN_WEIGHT":
        value = sys.maxsize
        for item in self._items:
          if self._knapsack.canPack(item) and item.getWeight() < value:
            selected = item
            value = item.getWeight()
        return selected
    else:
      raise Exception("Heuristic '" + heuristic + "' is not recognized by the system.")

In [ ]:
"""Implements a random hyper-heuristic"""
class RandomHH(HyperHeuristic):
  
  def __init__(self, features, heuristics):
    super().__init__(features, heuristics)    

  def getHeuristic(self, problem : Problem) -> str:    
    index = cPoint = numpy.random.randint(0, len(self._heuristics))
    #print("\t selects heuristic '" + self._heuristics[index] + "'")
    return self._heuristics[index]

In [ ]:
instance = "ks_45_0"

problem = KP(instance)
problem.solve("DEFAULT")
print("Solving with heuristic 'DEFAULT': " + str(problem.getObjValue()))

problem = KP(instance)
problem.solve("MAX_PROFIT")
print("Solving with heuristic 'MAX_PROFIT': " + str(problem.getObjValue()))

problem = KP(instance)
problem.solve("MAX_PROFIT/WEIGHT")
print("Solving with heuristic 'MAX_PROFIT/WEIGHT':" + str(problem.getObjValue()))

problem = KP(instance)
problem.solve("MIN_WEIGHT")
print("Solving with heuristic 'MIN_WEIGHT':" + str(problem.getObjValue()))

## Results may change at each run since we did not specify a seed for the random number generator
problem = KP(instance)
hh = RandomHH([], ["DEFAULT", "MAX_PROFIT", "MAX_PROFIT/WEIGHT", "MIN_WEIGHT"])
problem.solveHH(hh)
print("Solving with a random hyper-heuristic: " + str(problem.getObjValue()))

Solving with heuristic 'DEFAULT': -149508
Solving with heuristic 'MAX_PROFIT': -140232
Solving with heuristic 'MAX_PROFIT/WEIGHT':-150818
Solving with heuristic 'MIN_WEIGHT':-150102
Solving with a random hyper-heuristic: -146582
